In [1]:
import itertools
import random
import pandas as pd
import numpy as np
import json

In [2]:
matches = pd.read_excel("data/matches.xlsx", dtype={"home_team_id": str, "away_team_id": str}, index_col=0)
matches

,round,date,home_team_id,home_goals,home_shootout_goals,away_team_id,away_goals,away_shootout_goals
match_id,,,,,,,,
1,1,2023-01-01,08,2.0,NaN,12,4.0,NaN
2,1,2023-01-01,04,2.0,NaN,03,1.0,NaN
3,1,2023-01-01,02,5.0,NaN,06,0.0,NaN
4,1,2023-01-01,05,5.0,NaN,11,3.0,NaN
5,1,2023-01-01,01,1.0,NaN,10,5.0,NaN
...,...,...,...,...,...,...,...,...
62,11,2023-03-19,11,NaN,NaN,09,NaN,NaN
63,11,2023-03-19,07,NaN,NaN,02,NaN,NaN
64,11,2023-03-19,06,NaN,NaN,01,NaN,NaN


In [3]:
played_games = matches.dropna(subset=['home_goals', 'away_goals'])
played_games

,round,date,home_team_id,home_goals,home_shootout_goals,away_team_id,away_goals,away_shootout_goals
match_id,,,,,,,,
1,1,2023-01-01,08,2.0,NaN,12,4.0,NaN
2,1,2023-01-01,04,2.0,NaN,03,1.0,NaN
3,1,2023-01-01,02,5.0,NaN,06,0.0,NaN
4,1,2023-01-01,05,5.0,NaN,11,3.0,NaN
5,1,2023-01-01,01,1.0,NaN,10,5.0,NaN
6,1,2023-01-01,09,4.0,NaN,07,2.0,NaN
7,2,2023-01-08,03,3.0,NaN,08,5.0,NaN
8,2,2023-01-08,10,3.0,NaN,02,2.0,NaN
9,2,2023-01-08,12,3.0,NaN,01,2.0,NaN


In [4]:
unplayed_games = matches[matches['home_goals'].isna() & matches['away_goals'].isna()]
unplayed_games

,round,date,home_team_id,home_goals,home_shootout_goals,away_team_id,away_goals,away_shootout_goals
match_id,,,,,,,,
55,10,2023-03-12,01,NaN,NaN,07,NaN,NaN
56,10,2023-03-12,02,NaN,NaN,11,NaN,NaN
57,10,2023-03-12,04,NaN,NaN,12,NaN,NaN
58,10,2023-03-12,09,NaN,NaN,03,NaN,NaN
59,10,2023-03-12,10,NaN,NaN,06,NaN,NaN
60,10,2023-03-12,05,NaN,NaN,08,NaN,NaN
61,11,2023-03-19,12,NaN,NaN,10,NaN,NaN
62,11,2023-03-19,11,NaN,NaN,09,NaN,NaN
63,11,2023-03-19,07,NaN,NaN,02,NaN,NaN


In [5]:
# Initialize the classification dictionary
classification = {}

# Loop over each row in the DataFrame
for i, row in played_games.iterrows():
    home_team_id = row['home_team_id']
    away_team_id = row['away_team_id']
    home_goals = row['home_goals']
    away_goals = row['away_goals']
    home_shootout_goals = np.nan_to_num(row['home_shootout_goals'])
    away_shootout_goals = np.nan_to_num(row['away_shootout_goals'])
    
    # Initialize each team
    if home_team_id not in classification:
        classification[home_team_id] = {'W': 0, 'L': 0, 'GD': 0}
    if away_team_id not in classification:
        classification[away_team_id] = {'W': 0, 'L': 0, 'GD': 0}
        
    # Update the wins and loses for both teams
    if home_goals > away_goals or home_shootout_goals > away_shootout_goals:
        classification[home_team_id]['W'] += 1
        classification[away_team_id]['L'] += 1
    else:
        classification[home_team_id]['L'] += 1
        classification[away_team_id]['W'] += 1
    
    # Update the goal difference
    classification[home_team_id]['GD'] += home_goals - away_goals
    classification[away_team_id]['GD'] += away_goals - home_goals


    
# Sort the classification dictionary by the most wins and then by goal difference in case of a tie
classification = dict(sorted(classification.items(), key=lambda item: (-item[1]['W'], -item[1]['GD'])))

# Print the classification dictionary
classification

{'09': {'W': 7, 'L': 2, 'GD': 8.0},
 '10': {'W': 7, 'L': 2, 'GD': 7.0},
 '07': {'W': 6, 'L': 3, 'GD': 9.0},
 '04': {'W': 6, 'L': 3, 'GD': -1.0},
 '02': {'W': 5, 'L': 4, 'GD': 11.0},
 '05': {'W': 5, 'L': 4, 'GD': -1.0},
 '12': {'W': 4, 'L': 5, 'GD': -2.0},
 '11': {'W': 4, 'L': 5, 'GD': -2.0},
 '01': {'W': 4, 'L': 5, 'GD': -4.0},
 '06': {'W': 3, 'L': 6, 'GD': 0.0},
 '08': {'W': 3, 'L': 6, 'GD': -7.0},
 '03': {'W': 0, 'L': 9, 'GD': -18.0}}

In [6]:
unplayed_games_list = unplayed_games[['home_team_id', 'away_team_id']].apply(tuple, axis=1).to_list()

# Define a function to simulate all possible results
def simulate_results():
    for results in itertools.product(['W', 'L'], repeat=len(unplayed_games_list)):
        # Initialize a dictionary to keep track of the results
        result = {team: {'W': classification[team]['W'], 'L': classification[team]['L'], 'GD': classification[team]['GD']} for team in classification}
        # Update the dictionary based on the simulated results
        for i, (team1, team2) in enumerate(unplayed_games_list):
            if results[i] == 'W':
                result[team1]['W'] += 1
                result[team2]['L'] += 1
            else:
                result[team1]['L'] += 1
                result[team2]['W'] += 1
        # Calculate the positions of the teams based on their updated records
        positions = sorted(classification.keys(), key=lambda team: (-result[team]['W'], -result[team]['GD']))
        # Yield the positions as a tuple
        yield tuple(positions)

In [7]:
# Initialize a dictionary to keep track of the counts of each position for each team
counts = {team: {position: 0 for position in range(1, len(classification)+1)} for team in classification.keys()}

# Simulate all possible results and count the positions of each team
for positions in simulate_results():
    for i, team in enumerate(positions):
        counts[team][i+1] += 1

In [8]:
# Print the counts for each team and position
for team in counts:
    print(f'{team}:')
    for position, count in counts[team].items():
        print(f'  Position {position}: {count} times, {round(count/2**len(unplayed_games)*100, 2)}%')

09:
  Position 1: 2200 times, 53.71%
  Position 2: 992 times, 24.22%
  Position 3: 656 times, 16.02%
  Position 4: 224 times, 5.47%
  Position 5: 24 times, 0.59%
  Position 6: 0 times, 0.0%
  Position 7: 0 times, 0.0%
  Position 8: 0 times, 0.0%
  Position 9: 0 times, 0.0%
  Position 10: 0 times, 0.0%
  Position 11: 0 times, 0.0%
  Position 12: 0 times, 0.0%
10:
  Position 1: 1152 times, 28.12%
  Position 2: 1632 times, 39.84%
  Position 3: 992 times, 24.22%
  Position 4: 288 times, 7.03%
  Position 5: 32 times, 0.78%
  Position 6: 0 times, 0.0%
  Position 7: 0 times, 0.0%
  Position 8: 0 times, 0.0%
  Position 9: 0 times, 0.0%
  Position 10: 0 times, 0.0%
  Position 11: 0 times, 0.0%
  Position 12: 0 times, 0.0%
07:
  Position 1: 648 times, 15.82%
  Position 2: 864 times, 21.09%
  Position 3: 1008 times, 24.61%
  Position 4: 672 times, 16.41%
  Position 5: 712 times, 17.38%
  Position 6: 192 times, 4.69%
  Position 7: 0 times, 0.0%
  Position 8: 0 times, 0.0%
  Position 9: 0 times, 0.

In [9]:
save_file = open("results/positions_J9.json", "w")  
json.dump(counts, save_file)  
save_file.close()  